# 2021/10/31
# 本代码计算MSE，使用资料为MERRA2

In [1]:
import os
import sys
import xarray as xr
import numpy as np
module_path = ["/home/sun/mycode/module/","/data5/2019swh/mycode/module/"]
sys.path.append(module_path[0])
from module_sun import *
import metpy.calc as mpcalc

In [2]:
def cal_mse(T,q,z):
    cp  =  1.004 #kj/kg*K
    lv  =  2.5e3 #kj/kg
    g   =  9.8 #m/s^2

    mse  =  cp*T+lv*q+9.8*z/1000

    return mse


# 计算年的每日气候态MSE

path  =  '/home/sun/data/merra2_multi/'
path2 =  '/home/sun/data/merra2_specific/'
f0  =  xr.open_dataset(path+'0616.climate.nc')
file_list  =  os.listdir(path) ; file_list.sort()
file_list2 =  os.listdir(path2); file_list2.sort()

mse  =  np.zeros((365,f0.H.data.shape[1],f0.H.data.shape[2],f0.H.data.shape[3]))

for i in range(0,365):
    f1  =  xr.open_dataset(path+file_list[i])
    f2  =  xr.open_dataset(path2+file_list2[i])
    mse[i]  =  cal_mse(f1.T.data[0],f2.specific_humidity.data/1000,f1.H.data[0])


ncfile  =  xr.Dataset(
    {
        "mse": (["time", "lev","lat", "lon"], mse),
    },
    coords={
        "lon": (["lon"], f0.lon.data),
        "lat": (["lat"], f0.lat.data),
        "lev": (["lev"], f0.lev.data),
        "time": (["time"], np.linspace(1,365,365)),
    },
)
ncfile["lat"].attrs  =  f0["lat"].attrs
ncfile["lon"].attrs  =  f0["lon"].attrs
ncfile["lev"].attrs  =  f0["lev"].attrs
ncfile["mse"].attrs["units"]  =  "kj/kg"

ncfile.to_netcdf("/home/sun/data/merra2_climate_year_mse.nc")

In [3]:
from scipy import integrate
import numpy as np
import xarray as xr

path1  =  "/home/sun/data/merra2_specific/"
f0     =  xr.open_dataset(path1+"0227.climate_specific_humidity.nc")
f1     =  xr.open_dataset("/home/sun/data/merra2_climate_year_mse.nc")
file_list  =  os.listdir(path1)  ;  file_list.sort()
#
# 处理成侯平均
pentad_mse  =  np.zeros((73,f0.specific_humidity.data.shape[0],f0.specific_humidity.data.shape[1],f0.specific_humidity.data.shape[2]))
for pp in range(0,73):
    pentad_mse[pp,:]  +=  np.average(f1.mse.data[pp*5:pp*5+5,:],axis=0)
#
ncfile  =  xr.Dataset(
    {
        "mse": (["time", "lev","lat", "lon"], pentad_mse),
    },
    coords={
        "lon": (["lon"], f0.lon.data),
        "lat": (["lat"], f0.lat.data),
        "lev": (["lev"], f0.lev.data),
        "time": (["time"], np.linspace(1,73,73)),
    },
)
ncfile["lat"].attrs  =  f0["lat"].attrs
ncfile["lon"].attrs  =  f0["lon"].attrs
ncfile["lev"].attrs  =  f0["lev"].attrs
ncfile["mse"].attrs["units"]  =  "kj/kg"
#
ncfile.to_netcdf("/home/sun/data/merra2_climate_pentad_mse.nc")

In [7]:
path  =  "/home/sun/data/"
f     =  xr.open_dataset(path+"merra2_climate_pentad_mse.nc").sel(lev=slice(1000,300))
lev   =  f.lev.data

In [5]:
integrate_mse   =   np.zeros((f.mse.data.shape[0],f.mse.data.shape[2],f.mse.data.shape[3]))
for pen in range(0,f.mse.data.shape[0]):
    for lat in range(0,f.mse.data.shape[2]):
        for lon in range(0,f.mse.data.shape[3]):
            trans  =  np.nan_to_num(f.mse.data[pen,:,lat,lon],nan=0)
            integrate_mse[pen,lat,lon]  =  -1*integrate.trapz(trans,lev)/9.8

In [8]:
ncfile  =  xr.Dataset(
    {
        "integrated_mse": (["time", "lat", "lon"], integrate_mse),
    },
    coords={
        "lon": (["lon"], f.lon.data),
        "lat": (["lat"], f.lat.data),
        "time": (["time"], np.linspace(1,73,73)),
    },
)
ncfile["lat"].attrs  =  f["lat"].attrs
ncfile["lon"].attrs  =  f["lon"].attrs
ncfile["integrated_mse"].attrs["units"]  =  "kj/kg"

ncfile.to_netcdf("/home/sun/data/merra2_integrated_mse.nc")

PermissionError: [Errno 13] Permission denied: b'/home/sun/data/merra2_integrated_mse.nc'